In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
# Создание SparkSession
spark = SparkSession.builder.appName("weather").config('spark.master', 'local[*]').getOrCreate()

24/07/23 16:34:21 WARN Utils: Your hostname, themjdex-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/07/23 16:34:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/23 16:34:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('data/weather_data.csv', header=True, inferSchema=True)

In [4]:
df.printSchema()

root
 |-- station_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- temperature: double (nullable = true)
 |-- precipitation: double (nullable = true)
 |-- wind_speed: double (nullable = true)



In [5]:
df.count()

1000

In [6]:
# Найдите топ-5 самых жарких дней за все время наблюдений
df.select('date', 'temperature').orderBy(df.temperature.desc()).show(5)

+----------+------------------+
|      date|       temperature|
+----------+------------------+
|2021-08-20|39.982828249354846|
|2023-12-02| 39.96797489293784|
|2022-03-28|  39.8246894248997|
|2019-02-11| 39.76737697836647|
|2020-06-10| 39.69147838355929|
+----------+------------------+
only showing top 5 rows



In [7]:
# Найдите метеостанцию с наибольшим количеством осадков за последний год
df.select('station_id', 'precipitation'). \
    filter((df['date'] >= '2023-01-01') & (df['date'] <= '2023-12-31')). \
    groupBy('station_id').sum(). \
    sort(F.desc('sum(precipitation)')). \
    show(1)

+----------+------------------+
|station_id|sum(precipitation)|
+----------+------------------+
| station_5| 642.9302626767898|
+----------+------------------+
only showing top 1 row



In [8]:
# Подсчитайте среднюю температуру по месяцам за все время наблюдений
df.groupBy(F.month('date').alias('month')).avg('temperature').sort('month').show()

+-----+------------------+
|month|  avg(temperature)|
+-----+------------------+
|    1|11.356518462550754|
|    2| 9.067229891101926|
|    3| 7.244080205633994|
|    4|12.024529009744693|
|    5| 9.902883346912718|
|    6|13.421092297254138|
|    7|6.1857183016954576|
|    8|  10.9678002814186|
|    9| 9.596744236573942|
|   10|  9.09884344821895|
|   11| 7.265889994697494|
|   12|11.218592100674337|
+-----+------------------+



In [9]:
spark.stop()